In [52]:
import rdflib

# define an ontology with RDF triples in TTL ("turtle") format
ontology_data = """
prefix : <http://sites.psu.edu/reinhartgroup/mykg/>
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix unit: <http://qudt.org/vocab/unit/> .

# these classes do nothing, they are for example purposes only
:Measurement a rdfs:Class .
:Length rdfs:subClassOf :Measurement .
:Height rdfs:subClassOf :Measurement .
"""
ont_graph = rdflib.Graph()
ont_graph.parse(data=ontology_data, format="turtle")

# define constraint shapes using RDF triples in TTL
shacl_data = """
prefix : <http://sites.psu.edu/reinhartgroup/mykg/>
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix unit: <http://qudt.org/vocab/unit/> .

:myRule a sh:NodeShape ;             # enforce a Shape on the Node with type Measurement (name <myRule> can be anything)
    sh:targetClass :Measurement ;    # this should apply to :Measurement and all its subClasses
    sh:property [
        sh:path qudt:numericValue ;  # place constraints on the numericValue of the :Measurement
        sh:nodeKind sh:Literal ;     # make sure we are evaluating a literal
        sh:or ( [ sh:datatype xsd:double ] [ sh:datatype xsd:float ] [ sh:datatype xsd:decimal ] ) ;  # enforce certain datatypes
        sh:minInclusive 0.0 ;        # enforce >=0 values
        sh:minCount 1 ;              # enforce at least 1 qudt:numericValue
        sh:maxCount 1 ;              # enforce at most 1 qudt:numericValue
    ] .
"""
shacl_graph = rdflib.Graph()
shacl_graph.parse(data=shacl_data, format="turtle")

<Graph identifier=N6c567a686ea742da90920765cf897ece (<class 'rdflib.graph.Graph'>)>

In [57]:
import ontology
import importlib
importlib.reload(ontology)

# define data instances using RDF triples in TTL
instances = [
    # should fail due to negative value, violating minInclusive
    dict(name='Length1', type='Length', value=-1.0, unit='CentiM'),
    # should pass
    dict(name='Length2', type='Length', value=2.0, unit='M'),
    # should fail due to wrong datatype
    dict(name='Height1', type='Height', value='"3"^^xsd:str', unit='M'),
    # should pass
    dict(name='Height2', type='Height', value=4.0, unit='M'),
]

data = ontology.DataGraphTemplate(instances)
print(data)


prefix : <http://sites.psu.edu/reinhartgroup/mykg/>
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Length1 a :Length ;
  qudt:numericValue -1.0 ;
  qudt:unit unit:CentiM ;
.

:Length2 a :Length ;
  qudt:numericValue 2.0 ;
  qudt:unit unit:M ;
.

:Height1 a :Height ;
  qudt:numericValue "3"^^xsd:str ;
  qudt:unit unit:M ;
.

:Height2 a :Height ;
  qudt:numericValue 4.0 ;
  qudt:unit unit:M ;
.



In [59]:
shacl = ontology.Validator(ont_graph, shacl_graph)
bad_nodes = shacl.run(data.as_graph())
print(bad_nodes)

['Height1', 'Length1']
